# Common Task1. Electron/photon classification

Datasets: 
* https://cernbox.cern.ch/index.php/s/AtBT8y4MiQYFcgc (photons) 

* https://cernbox.cern.ch/index.php/s/FbXw3V4XNyYB3oA (electrons) 

Description: 

32x32 matrices (two channels - hit energy and time) for two classes of particles electrons and photons impinging on a calorimeter 
Please use a deep learning method of your choice to achieve the highest possible classification on this dataset (we ask that you do it both in Keras/Tensorflow and in PyTorch). Please provide a Jupyter notebook that shows your solution. The model you submit should have a ROC AUC score of at least 0.80. 

## Data Preparation

In [1]:
# First, expolore the data format in hdf5 file
import h5py
import numpy as np
f1 = h5py.File('../input/electron-photon/download', 'r')
f2 = h5py.File('../input/electron-photon/download_1', 'r')

Electron_X = np.array(f1['X'])
Electron_y = np.array(f1['y'])
Parton_X = np.array(f2['X'])
Parton_y = np.array(f2['y'])
print(Electron_X.shape, Electron_y.shape, Parton_X.shape, Parton_y.shape)

(249000, 32, 32, 2) (249000,) (249000, 32, 32, 2) (249000,)


In [2]:
All_X = np.concatenate((Electron_X, Parton_X), axis=0)
All_y = np.concatenate((Electron_y, Parton_y), axis=0)
# print(All_X.shape, All_y.shape)
rand_seed = 12
index = np.random.permutation(len(All_y))
# here the dataset is flattened
All_X, All_y = All_X[index][:,:,:,0].reshape((-1,32*32)), All_y[index]
print(All_X.shape, All_y.shape)

# clear cache to save memory
del Electron_X, Electron_y, Parton_X, Parton_y

(498000, 1024) (498000,)


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(All_X, All_y, test_size=0.2, random_state=12)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

del All_X, All_y

(398400, 1024) (99600, 1024)
(398400,) (99600,)


## Version 1: Pytorch MLP

After reading paper https://arxiv.org/abs/1807.11916, I decided to start with a simple MLP. 

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.transforms as transforms
from tqdm import tqdm
import random

In [5]:
# set seed to be able to get reproducible results
SEED = 293
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

In [6]:
data_transform = transforms.Compose([transforms.ToTensor()])

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)

train_set = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train.reshape((-1,1))))
valid_set = TensorDataset(torch.from_numpy(X_valid), torch.from_numpy(y_valid.reshape((-1,1))))
test_set = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test.reshape((-1,1))))

train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=32, shuffle=False)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [8]:
print(len(X_train), train_set.__len__(),
len(X_valid), valid_set.__len__(),
len(X_test), test_set.__len__())

318720 318720 79680 79680 99600 99600


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_stack = nn.Sequential(
            # layer 1
            nn.Linear(32*32, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            # layer 2
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            # layer 3
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            # layer 4
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            # output layer
            nn.Linear(256,1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        logits = self.linear_stack(x)
        return logits

In [10]:
model = MLP().to(device)
print(model)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
epochs = 30
min_valid_loss = np.inf

for e in range(epochs):
    train_loss = 0.0
    train_correct = 0
    model.train()
    for data, labels in tqdm(train_loader):
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
         
        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = model(data)    
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()
        # Calculate Correct
        train_correct += ((target>0.5).float() == labels).sum().item()
    scheduler.step()
    
    valid_loss = 0.0
    val_correct = 0
    model.eval()     # Optional when not using Model Specific layer
    for data, labels in valid_loader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        # Forward Pass
        target = model(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
        # Calculate Right Prediction
        val_correct += ((target>0.5).float() == labels).sum().item()
 
    print('Epoch: {}: \t Training Loss:{:.6f}\t Training Accuracy:{:.6f} \t Validation Loss:{:.6f} \t Validation Accuracy:{:.6f}'.format(
        e+1, train_loss / len(train_loader), train_correct*1.0 / len(X_train), valid_loss / len(valid_loader), val_correct*1.0 / len(X_valid)
    ))
     
    if min_valid_loss > valid_loss:
        min_valid_loss = valid_loss
         
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=1024, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=256, out_features=1, bias=True)
    (13): Sigmoid()
  )
)


100%|██████████| 9960/9960 [00:30<00:00, 324.34it/s]


Epoch: 1: 	 Training Loss:0.621354	 Training Accuracy:0.664263 	 Validation Loss:0.590146 	 Validation Accuracy:0.693750


100%|██████████| 9960/9960 [00:30<00:00, 326.85it/s]


Epoch: 2: 	 Training Loss:0.595032	 Training Accuracy:0.694741 	 Validation Loss:0.584734 	 Validation Accuracy:0.695833


...

Epoch: 28: 	 Training Loss:0.551858	 Training Accuracy:0.729239 	 Validation Loss:0.547503 	 Validation Accuracy:0.729882


100%|██████████| 9960/9960 [00:31<00:00, 316.78it/s]


Epoch: 29: 	 Training Loss:0.551375	 Training Accuracy:0.730240 	 Validation Loss:0.547793 	 Validation Accuracy:0.730183


100%|██████████| 9960/9960 [00:31<00:00, 315.87it/s]


Epoch: 30: 	 Training Loss:0.551613	 Training Accuracy:0.729954 	 Validation Loss:0.547539 	 Validation Accuracy:0.729719


In [11]:
import gc
def test(model, test_loader):
    total = 0
    correct = 0
    model.eval()     # Optional when not using Model Specific layer
    y_pred = np.array([])
    with torch.no_grad():
        for data, labels in test_loader:
            # Transfer Data to GPU if available
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()
            # Forward Pass
            target = model(data)
            # Calculate Right Prediction
            total += labels.size(0)
            correct += ((target>0.5).float() == labels).sum().item()
            # Save prediction
            y_pred = np.append(y_pred, target.cpu().detach().numpy())
    gc.collect()
    print('Testing Accuracy:{:.6f}'.format(correct*1.0 / total))
    return y_pred

In [12]:
best_model = MLP()
best_model.load_state_dict(torch.load("saved_model.pth"))
best_model.to(device)
y_pred = test(best_model, test_loader)

Testing Accuracy:0.726968


In [13]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.7928410294176385